In [1]:
# Install Packages
from selenium import webdriver    # pip install selenium 
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup    # pip install beautifulsoup4

# Deal with Infinite Scrolling Page

In order to use selenium, we need a web driver.

In [2]:
# Install Chrome Driver
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\liang\AppData\Local\Temp/ipykernel_12232/439201904.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
# Open Reddit search page by the keyword you specified
keyword = 'spx'
base_url = "https://www.reddit.com/search/?q="
driver.get(f"{base_url}{keyword}")

# waiting for the page to load
time.sleep(3)

# Get the height of the searched results
previous_height = driver.execute_script('return document.body.scrollHeight')

Now, we need to scroll to the bottom.If the new page 

In [4]:
# scroll to the bottom
while True:
    # Execute the scrolling
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # waiting for the page to load
    time.sleep(3)
    # get the new height of the searched results
    new_height = driver.execute_script('return document.body.scrollHeight')
    # if the height doesn't change, it means we have scrolled to the bottom
    if new_height == previous_height:
        break
    previous_height = new_height

The page is now scrolled to the bottom. As the page is completely loaded, we will scrape the data we want.

# Extract Data

In [5]:
# create list variable to store the extracted data
subredditList = []
timestampList = []
titleList = []
upvoteList = []
numberOfCommentList = []
idList = []
urlList = []

In [6]:
# Store the source code of the web page in a variable
source = driver.page_source

# Now using beautiful soup
soup = BeautifulSoup(source, 'html.parser')

In [7]:
# Find the source code for each post
posts = soup.find_all("faceplate-tracker", {"source": "search",
                                              "action": "view",
                                              "noun": "post",
                                              "data-testid": "search-post"
                                              })

In [8]:
# Extract the information we need
for post in posts:
    subredditList.append(post.find("a", 
                          {"class": "flex items-center text-neutral-content-weak font-semibold"}).text.strip().split('/')[1])
    

    timestampList.append(post.find("faceplate-timeago")['ts'])

    titleList.append(post.find('a', {"data-testid": "post-title"}).text.split('\n')[1].strip())

    upvoteList.append(post.find("div", {"class": "text-neutral-content-weak text-12"}).find_all('faceplate-number')[0]['number'])

    numberOfCommentList.append(post.find("div", {"class": "text-neutral-content-weak text-12"}).find_all('faceplate-number')[1]['number'])

    idList.append(post.find("a", 
                    {"class": 
                      "text-16 xs:text-18 line-clamp-3 text-ellipsis text-neutral-content font-semibold mb-xs "
                      "no-underline hover:no-underline"})['id'].split('-')[-1]
                  )
    
    urlList.append(post.find("a", 
                    {"class": 
                      "text-16 xs:text-18 line-clamp-3 text-ellipsis text-neutral-content font-semibold mb-xs "
                      "no-underline hover:no-underline"})['href']
                    )

In [9]:
# Store the information into a Data Frame
data = pd.DataFrame({"subreddit": subredditList,
                      "timestamp": timestampList,
                      "title": titleList,
                      "upvote": upvoteList,
                      "number_of_comments": numberOfCommentList,
                      "id": idList,
                      "url": urlList
    })

In [10]:
# Create a data column
data['date'] = data['timestamp'].str[:10] 

In [11]:
data

,subreddit,timestamp,title,upvote,number_of_comments,id,url,date
0,options,2023-05-21T18:51:21.053000+0000,SPX trading experience?,18,43,t3_13o2zii,/r/options/comments/13o2zii/spx_trading_experi...,2023-05-21
1,options,2022-12-25T09:47:27.413000+0000,SPX??,27,46,t3_zuv2cd,/r/options/comments/zuv2cd/spx/,2022-12-25
2,interactivebrokers,2023-02-19T16:15:36.416000+0000,spx trading,0,12,t3_116gl1w,/r/interactivebrokers/comments/116gl1w/spx_tra...,2023-02-19
3,wallstreetbets,2022-10-14T16:42:13.291000+0000,I discovered SPX options,182,63,t3_y3yvb2,/r/wallstreetbets/comments/y3yvb2/i_discovered...,2022-10-14
4,wallstreetbets,2022-09-13T22:21:35.578000+0000,Turned 1.6K to 57K in 2 hours with SPX 0 Day e...,10175,961,t3_xdkigc,/r/wallstreetbets/comments/xdkigc/turned_16k_t...,2022-09-13
...,...,...,...,...,...,...,...,...
240,ModernWarfareII,2023-01-12T04:12:47.542000+0000,Spx-80 real life variant?,5,8,t3_109pww0,/r/ModernWarfareII/comments/109pww0/spx80_real...,2023-01-12
241,thetagang,2022-02-17T04:15:41.347000+0000,Anybody selling SPX puts? Looks like easy premium,0,36,t3_sufnz9,/r/thetagang/comments/sufnz9/anybody_selling_s...,2022-02-17
242,fidelityinvestments,2022-06-13T17:20:02.214000+0000,"SPX ""volume"" on Active Trader Pro",3,1,t3_vbhand,/r/fidelityinvestments/comments/vbhand/spx_vol...,2022-06-13
243,RobinHood,2019-01-23T14:49:29.962000+0000,I cannot find the SPX index and same expiratio...,0,4,t3_aj0ezn,/r/RobinHood/comments/aj0ezn/i_cannot_find_the...,2019-01-23
